In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from scipy.linalg import eigh
from matplotlib import style
import mpl_axes_aligner
import seaborn as sns
import pandas as pd
import scienceplots
import numpy as np

In [ ]:
plt.style.use(['science', 'nature'])

In [ ]:
t1_df = pd.read_csv('../stats/preprocessed_data.csv')

In [ ]:
""" t1_df = t1_df\
    .drop(columns=t1_df.filter(like='volume_lh_pial'))\
    .drop(columns=t1_df.filter(like='volume_rh_pial'))\
    .drop(columns=t1_df.filter(like='thickness_lh_pial'))\
    .drop(columns=t1_df.filter(like='thickness_rh_pial'))\
    .drop(columns=["etiv_volume_lh",
                   "etiv_volume_rh",
                   "etiv_meancurv_lh",
                   "etiv_meancurv_rh",
                   "etiv_thickness_lh",
                   "etiv_thickness_rh",
                   "brainsegvolnotvent_volume_lh",
                   "brainsegvolnotvent_volume_rh",
                   "brainsegvolnotvent_thickness_lh",
                   "brainsegvolnotvent_thickness_rh",
                   "brainsegvolnotvent_meancurv_lh",
                   "brainsegvolnotvent_meancurv_rh",
                   "meanthickness_thickness_lh",
                   "meanthickness_thickness_rh"])\
    .drop(columns=["etiv_meancurv_lh_pial", "etiv_meancurv_rh_pial", "brainsegvolnotvent_meancurv_lh_pial", "brainsegvolnotvent_meancurv_rh_pial"])
t1_df.to_csv('../stats/cleaned_data.csv', index=False) """

In [ ]:
volume_a2009s_left = t1_df.filter(like='volume_lh_a2009s').assign(group=t1_df['group'])
meancurv_a2009s_left = t1_df.filter(like='meancurv_lh_a2009s').assign(group=t1_df['group'])
thickness_a2009s_left = t1_df.filter(like='thickness_lh_a2009s').assign(group=t1_df['group'])

volume_a2009s_right = t1_df.filter(like='volume_rh_a2009s').assign(group=t1_df['group'])
meancurv_a2009s_right = t1_df.filter(like='meancurv_rh_a2009s').assign(group=t1_df['group'])
thickness_a2009s_right = t1_df.filter(like='thickness_rh_a2009s').assign(group=t1_df['group'])

In [ ]:
volume_pial_left = t1_df.filter(like='volume_lh_pial').assign(group=t1_df['group'])
meancurv_pial_left = t1_df.filter(like='meancurv_lh_pial').assign(group=t1_df['group'])
thickness_pial_left = t1_df.filter(like='thickness_lh_pial').assign(group=t1_df['group'])

volume_pial_right = t1_df.filter(like='volume_rh_pial').assign(group=t1_df['group'])
meancurv_pial_right = t1_df.filter(like='meancurv_rh_pial').assign(group=t1_df['group'])
thickness_pial_right = t1_df.filter(like='thickness_rh_pial').assign(group=t1_df['group'])

In [ ]:
volume_left = t1_df.filter(regex='volume_lh$').assign(group=t1_df['group'])
meancurv_left = t1_df.filter(regex='meancurv_lh$').assign(group=t1_df['group'])
thickness_left = t1_df.filter(regex='thickness_lh$').assign(group=t1_df['group'])

volume_right = t1_df.filter(regex='volume_rh$').assign(group=t1_df['group'])
meancurv_right = t1_df.filter(regex='meancurv$_rh').assign(group=t1_df['group'])
thickness_right = t1_df.filter(regex='thickness_rh$').assign(group=t1_df['group'])

In [ ]:
aseg_df = t1_df.iloc[:,-59:]

In [ ]:
def plot_catplot(df, quantiles, column='aseg', hue=None, height=10, aspect=.8):
    
    quantile_values = df[column].quantile(quantiles)
    df['range'] = pd.cut(df[column],
                              bins=quantile_values,
                              include_lowest=True)

    g = sns.catplot(
        x=column, y='ROI', kind='box',
        hue=hue, 
        data=df,
        height=height,
        aspect=aspect,
        col='range',
        col_wrap=2,
        sharex=False,
        sharey=False,
    )

    g.set_titles("")

    plt.show()

In [ ]:
def biplot(df_scores, df_loadings, pc1, pc2, ax, num_features=5):

    loadings_magnitude = df_loadings.pow(2).sum(axis=0)
    top_features = loadings_magnitude.nlargest(num_features).index
    df_loadings_limited = df_loadings[top_features]

    ax.scatter(df_scores[pc1], df_scores[pc2], color='b', alpha=0.5)
    ax.set_xlabel(pc1, fontsize=10)
    ax.set_ylabel(pc2, fontsize=10)

    ax2 = ax.twinx().twiny()

    x_lim, y_lim = ax.get_xlim(), ax.get_ylim()
    max_x, max_y = x_lim[1] - x_lim[0], y_lim[1] - y_lim[0]
    max_length = max(max_x, max_y) / 2
    norm_loadings = df_loadings_limited / df_loadings_limited.abs().max().max() * max_length

    for col in norm_loadings.columns:
        tipx = norm_loadings.loc[pc1, col]
        tipy = norm_loadings.loc[pc2, col]
        
        ax2.arrow(0, 0, tipx, tipy, color='r')

        #if tipx > 0:
        ax2.text(tipx, tipy, col, fontdict={'color': 'g', 'weight': 'bold', 'size': 8}, ha='center')
        #else:
            #ax2.text(tipx, tipy, col, fontdict={'color': 'g', 'weight': 'bold', 'size': 8}, ha='left')

    mpl_axes_aligner.align.xaxes(ax, 0, ax2, 0, 0.5)
    mpl_axes_aligner.align.yaxes(ax, 0, ax2, 0, 0.5)

In [ ]:
def plot_random_biplots(df_scores: pd.DataFrame, df_loadings: pd.DataFrame, num_plots: int = 6, grid_size=(3, 2)) -> None:

    num_components = df_scores.shape[1]
    random_pairs = np.random.choice(range(num_components), (num_plots, 2), replace=False)

    fig, axes = plt.subplots(grid_size[0], grid_size[1], figsize=(15, 15))
    axes = axes.flatten()

    for i, (pc1_idx, pc2_idx) in enumerate(random_pairs):

        pc1 = f"PC{pc1_idx+1}"
        pc2 = f"PC{pc2_idx+1}"

        biplot(df_scores, df_loadings, pc1, pc2, axes[i])

    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
def reshape_and_clean(df, value_name, parcellation_name, pattern):
    
    reshaped_df = pd.melt(
        df.reset_index(),
        id_vars=["index", "group"],
        var_name="ROI",
        value_name=value_name
    )

    reshaped_df["ROI"] = reshaped_df["ROI"].str.replace(pattern, "")
    reshaped_df["parcellation"] = parcellation_name
    return reshaped_df

In [ ]:
def plot_corr_matrix(df, vmax=None):

    corr = df.corr()
    mask = np.triu(np.ones_like(corr, dtype=bool))
    cmap = sns.diverging_palette(230, 20, as_cmap=True)
    plt.figure(figsize=(20, 20))
    sns.heatmap(corr, mask=mask, cmap=cmap, vmax=vmax, center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5})
    plt.show()

como nem todos os atributos possuem uma distribuição normal é importante normalizar os atributos para evitar problemas com métodos sensiveis a distribuição.  
devido a natureza dos dados os outiliers podem ser casos especificos e que são importantes para a analise e não devem ser removidos. modelos robustos a outliers como random forest e gradient boosting são ideais para esse caso  
devido a quantidade de atributos e a não linearidade uma redução com pca pode trazer beneficios

In [ ]:
aseg_long = pd.melt(
    aseg_df.reset_index(),
    id_vars=["index", "group"],
    var_name="ROI",
    value_name="aseg",
).drop(columns=["index"]).reset_index(drop=True)

In [ ]:
quantiles = [0.0, 0.25, 0.5, 0.75, 1.0]
plot_catplot(aseg_long, quantiles)

Desikan-Killiany com e sem Pial não possue diferença para as métricas de volume e thickness causando redundancia de valores.

In [ ]:
a2009s_left = reshape_and_clean(meancurv_a2009s_left, "meancurv", "Destrieux", "_meancurv_lh_a2009s")
a2009s_right = reshape_and_clean(meancurv_a2009s_right, "meancurv", "Destrieux", "_meancurv_rh_a2009s")
desikan_pial_left = reshape_and_clean(meancurv_pial_left, "meancurv", "Desikan-Killiany with Pial", "_meancurv_lh_pial")
desikan_pial_right = reshape_and_clean(meancurv_pial_right, "meancurv", "Desikan-Killiany with Pial", "_meancurv_rh_pial")
desikan_left = reshape_and_clean(meancurv_left, "meancurv", "Desikan-Killiany", "_meancurv_lh")
desikan_right = reshape_and_clean(meancurv_right, "meancurv", "Desikan-Killiany", "_meancurv_rh")

meancurv_df = pd.concat(
    [a2009s_left, a2009s_right, desikan_left, desikan_right, desikan_pial_left, desikan_pial_right]
).drop(columns=["index"]).reset_index(drop=True)

quantiles = [0.0, 0.5, 1.0]
plot_catplot(meancurv_df, quantiles, column='meancurv', hue='parcellation', height=25, aspect=.4)

In [ ]:
a2009s_left = reshape_and_clean(volume_a2009s_left, "volume", "Destrieux", "_volume_lh_a2009s")
a2009s_right = reshape_and_clean(volume_a2009s_right, "volume", "Destrieux", "_volume_rh_a2009s")
desikan_pial_left = reshape_and_clean(volume_pial_left, "volume", "Desikan-Killiany with Pial", "_volume_lh_pial")
desikan_pial_right = reshape_and_clean(volume_pial_right, "volume", "Desikan-Killiany with Pial", "_volume_rh_pial")
desikan_left = reshape_and_clean(volume_left, "volume", "Desikan-Killiany", "_volume_lh")
desikan_right = reshape_and_clean(volume_right, "volume", "Desikan-Killiany", "_volume_rh")

volume_df = pd.concat(
    [a2009s_left, a2009s_right, desikan_pial_left, desikan_pial_right, desikan_left, desikan_right]
).drop(columns=["index"]).reset_index(drop=True)

quantiles = [0.0, 0.5, 1.0]
plot_catplot(volume_df, quantiles, column='volume', hue='parcellation', height=25, aspect=.4)

In [ ]:
a2009s_left = reshape_and_clean(thickness_a2009s_left, "thickness", "Destrieux", "_thickness_lh_a2009s")
a2009s_right = reshape_and_clean(thickness_a2009s_right, "thickness", "Destrieux", "_thickness_rh_a2009s")
desikan_pial_left = reshape_and_clean(thickness_pial_left, "thickness", "Desikan-Killiany with Pial", "_thickness_lh_pial")
desikan_pial_right = reshape_and_clean(thickness_pial_right, "thickness", "Desikan-Killiany with Pial", "_thickness_rh_pial")
desikan_left = reshape_and_clean(thickness_left, "thickness", "Desikan-Killiany", "_thickness_lh")
desikan_right = reshape_and_clean(thickness_right, "thickness", "Desikan-Killiany", "_thickness_rh")

thickness_df = pd.concat(
    [a2009s_left, a2009s_right, desikan_pial_left, desikan_pial_right, desikan_left, desikan_right]
).drop(columns=["index"]).reset_index(drop=True)

quantiles = [0.0, 0.5, 1.0]
plot_catplot(thickness_df, quantiles, column='thickness', hue='parcellation', height=25, aspect=.4)

grande parte dos atributos mostram uma interdependencia podendo significar multicolinearidade. isso se deve principalmente por redundancia, por exemplo, total gray matter volume and supratentorial volume estão fortemente correlacionado porque um é um subconjunto ou derivado do outro. neste caso a seleção de atributos se torna importante.

In [ ]:
volume = t1_df.filter(like='volume')
volume_norm = (volume - volume.mean()) / volume.std()
corr_volume = volume_norm.corr()

meancurv = t1_df.filter(like='meancurv')
meancurv_norm = (meancurv - meancurv.mean()) / meancurv.std()
corr_meancurv = meancurv_norm.corr()

thickness = t1_df.filter(like='thickness')
thickness_norm = (thickness - thickness.mean()) / thickness.std()
corr_thickness = thickness_norm.corr()

In [ ]:
plot_corr_matrix(thickness_norm, vmax=.3)

In [ ]:
X = t1_df.drop(columns=['subject', 'group'])

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
cov = np.cov(X_scaled.T)

eigvals, eigvecs = eigh(cov)
indices = np.argsort(eigvals)[::-1]
eigvals, eigvecs = eigvals[indices], eigvecs[:, indices]

explained_variance = eigvals / eigvals.sum()
cumulative_variance = np.cumsum(explained_variance)

threshold = 0.8
n_components = np.argmax(cumulative_variance > threshold) + 1

pca_df = pd.DataFrame({
    'Principal Component': [f'{i+1}' for i in range(len(explained_variance[:n_components]))],
    'Explained Variance': explained_variance[:n_components],
    'Cumulative Explained Variance': cumulative_variance[:n_components]
})

plt.figure(figsize=(10, 6))
sns.barplot(x='Principal Component', y='Explained Variance', data=pca_df, color='lightblue', label='Explained Variance')
plt.step(pca_df['Principal Component'], pca_df['Cumulative Explained Variance'], where='mid', color='darkblue', linewidth=2, label='Cumulative Explained Variance')

plt.xlabel('Principal Component')
plt.ylabel('Variance Explained')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
pca = PCA(n_components=n_components)
X_pca = pca.fit_transform(X_scaled)

df_scores = pd.DataFrame(X_pca, columns=[f'PC{i+1}' for i in range(n_components)])
df_loadings = pd.DataFrame(pca.components_, columns=X.columns, index=[f'PC{i+1}' for i in range(n_components)])

In [ ]:
plot_corr_matrix(df_scores)

In [ ]:
plot_random_biplots(df_scores, df_loadings)

- logistic regression
- support vector machine
- random forest
- gradient boosting